In [1]:
# TTS
!pip install numba inflect psutil
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
!pip install transformers

# Redis
!pip install redis
!pip install rq
!pip install time

!pip install redis rq google-api-python-client google-auth-httplib2 google-auth-oauthlib


!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip install .

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 3.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1987, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1987 (delta 0), reused 1 (delta 0), pack-reused 1982 (from 1)
Receiving objects: 100% (1987/1987), 54.19 MiB | 37.59 MiB/s, done.
Resolving deltas: 100% (909/909), done.
/content/tortoise-tts
Processing /content/tortoise-tts
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 53.8 MB/s eta 0:00:

In [ ]:
%%writefile copy.py
def upload_to_drive(file_path: str):
    """Upload file to Google Drive and return shareable link"""
    try:
        # Use Google Colab's built-in authentication
        # auth.authenticate_user()

        # Create Drive API service
        drive_service = build('drive', 'v3')

        print("file_path: "+file_path)
        # File metadata
        file_metadata = {
            'name': os.path.basename("content/tortoise-tts/output/%s/random_0_0.wav"%(file_path)),
            # Optional: specify folder ID to upload to a specific folder
            # 'parents': ['folder_id_here']
        }

        print("path: "+"content/tortoise-tts/%s/random_0_0.wav"%(file_path))

        # Create media file
        media = MediaFileUpload(
            "content/tortoise-tts/%s/random_0_0.wav"%(file_path),
            mimetype='audio/wav',
            resumable=True
        )

        # Upload file
        file = drive_service.files().create(
            body=file_metadata,
            media_body=media,
            fields='id, webViewLink'
        ).execute()

        # Make the file publicly accessible
        drive_service.permissions().create(
            fileId=file['id'],
            body={'type': 'anyone', 'role': 'reader'},
            fields='id'
        ).execute()

        # Clean up local file
        os.remove(output_file)

        return {
            'file_id': file['id'],
            'view_link': file['webViewLink'],
            'direct_link': f"https://drive.google.com/uc?export=download&id={file['id']}"
        }

    except Exception as e:
        logger.error(f"Drive upload failed: {str(e)}")
        raise


In [12]:
# create tts_job.py

%%writefile tts_job.py
import subprocess
import logging
from pathlib import Path
from google.colab import auth

import uuid
import os
import uuid
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import time

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def run_tts_command(text, jobId, voice="random", preset="fast", output_dir="output"):
    """
    Execute TTS command with the given parameters

    Args:
        text (str): Text to convert to speech
        voice (str): Voice to use (default: random)
        preset (str): Preset to use (default: fast)
        output_dir (str): Directory to save output files

    Returns:
        dict: Result containing status and output file path
    """
    try:
        # Create output directory if it doesn't exist
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        # Generate unique filename
        output_file = f"{output_dir}/{jobId}"

        # Construct the command
        cmd = [
            "python",
            "tortoise/do_tts.py",
            "--text",
            text,
            "--voice",
            voice,
            "--preset",
            preset,
            "--output_path",
            output_file
        ]

        logger.info(f"Executing command: {' '.join(cmd)}")

        # Execute the command
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        # Get output and errors
        stdout, stderr = process.communicate()


        if process.returncode == 0:
            logger.info("TTS command executed successfully")

            return {
                "status": "success",
                "output_file": output_file,
                "stdout": stdout,
                "stderr": stderr
            }
        else:
            logger.error(f"TTS command failed: {stderr}")
            return {
                "status": "error",
                "error": stderr,
                "stdout": stdout
            }

    except Exception as e:
        logger.error(f"Error executing TTS command: {str(e)}")
        return {
            "status": "error",
            "error": str(e)
        }

Overwriting tts_job.py


In [9]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
# worker.py
import logging
from redis import Redis
from rq import Worker, Queue

def print_num(low,high):
    print("start")
    x=low
    while x<high:
        print(x)
        x+=1
    print("end")


# Set up logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Redis connection settings
REDIS_HOST = os.getenv("REDIS_HOST")
REDIS_PORT = os.getenv("REDIS_PORT")
REDIS_PASSWORD =  os.getenv("REDIS_PASSWORD")

def setup_worker():
    try:
        # Set up Redis connection
        redis_conn = Redis(
            host=REDIS_HOST,
            port=REDIS_PORT,
            password=REDIS_PASSWORD,
            decode_responses=False  # Important: Set to False for job serialization
        )

        # Test Redis connection
        redis_conn.ping()
        logger.info("Successfully connected to Redis")

        # Create queue
        queue = Queue(
            'task_queue',
            connection=redis_conn,
        )

        # Create worker with custom settings
        worker = Worker(
            [queue],
            connection=redis_conn,
        )
        logger.info("Worker created successfully")

        return worker

    except Exception as e:
        logger.error(f"Error occurred: {str(e)}")
        raise

if __name__ == '__main__':
    try:
        worker = setup_worker()
        logger.info("Starting worker...")
        worker.work(with_scheduler=True)
    except Exception as e:
        logger.error(f"Failed to start worker: {str(e)}")
        raise

INFO:rq.worker:Worker rq:worker:a457d873fd92481bb6302b51cca0cdf6 started with PID 989, version 2.0.0
INFO:rq.worker:Subscribing to channel rq:pubsub:a457d873fd92481bb6302b51cca0cdf6
INFO:rq.worker:*** Listening on task_queue...
INFO:rq.scheduler:Scheduler for task_queue started with PID 10608
INFO:rq.worker:Cleaning registries for queue: task_queue
INFO:rq.worker:task_queue: tts_job.run_tts_command('i am batman, bati bati batimaaaan,  bati bati batimaaaan', 'random', 'ultra_fast') (98c1d641-5ffa-46d7-80ba-a4bb5b4dba9a)


file_path: output/tts_8bea1260-1cc0-4d42-af63-3ef08230eb57.wav

path: content/tortoise-tts/output/tts_8bea1260-1cc0-4d42-af63-3ef08230eb57.wav/random_0_0.wav

ERROR:tts_job:Drive upload failed: [Errno 2] No such file or directory: 'content/tortoise-tts/output/tts_8bea1260-1cc0-4d42-af63-3ef08230eb57.wav/random_0_0.wav'
ERROR:tts_job:Error executing TTS command: [Errno 2] No such file or directory: 'content/tortoise-tts/output/tts_8bea1260-1cc0-4d42-af63-3ef08230eb57.wav/random_0_0.wav'
INFO:rq.worker:Successfully completed tts_job.run_tts_command('i am batman, bati bati batimaaaan,  bati bati batimaaaan', 'random', 'ultra_fast') job in 0:01:26.395383s on worker a457d873fd92481bb6302b51cca0cdf6
INFO:rq.worker:task_queue: Job OK (98c1d641-5ffa-46d7-80ba-a4bb5b4dba9a)
INFO:rq.worker:Result is kept for 500 seconds
INFO:rq.worker:Cleaning registries for queue: task_queue
INFO:rq.worker:Cleaning registries for queue: task_queue
INFO:rq.worker:Cleaning registries for queue: task_queue
INFO:rq.worker:Cleaning registries for queue: task_queue


file_path: /content/tortoise-tts/output/tts_5848d11a-d44d-4ba1-bdf1-80db62c22186.wav/random_0_0.wav


{'file_id': '1oluvne16yfDb7VQwDYUMJszGXB1bJwLv',
 'view_link': 'https://drive.google.com/file/d/1oluvne16yfDb7VQwDYUMJszGXB1bJwLv/view?usp=drivesdk',
 'direct_link': 'https://drive.google.com/uc?export=download&id=1oluvne16yfDb7VQwDYUMJszGXB1bJwLv'}

!pip install numba inflect psutil
!pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
!pip install transformers

!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip install .